In [1]:
#Source du code
#https://larevueia.fr/latent-dirichlet-allocation-topic-modeling-en-python/


In [2]:
#Import de pandas
import pandas as pd

In [3]:
#Installation de nltk et de gensim
!pip install nltk
!pip install gensim

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.0 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 337.6 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     --------------------------- ---------- 41.0/57.6 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 57.6/57.6 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   ---------- ----------------------------- 71.7/269.0 kB 1.3 MB/s eta 0:00:01
   --------------------- ------------------ 143.4/269.0 kB 1.4 MB/s eta 0:00:01
   ----------------------------------- ---- 235.5/269.0 

  error: subprocess-exited-with-error
  
  × Building wheel for gensim (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [714 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\gensim
      copying gensim\downloader.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\interfaces.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\matutils.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\nosy.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\utils.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\__init__.py -> build\lib.win-amd64-cpython-312\gensim
      creating build\lib.win-amd64-cpython-312\gensim\corpora
      copying gensim\corpora\bleicorpus.py -> build\lib.win-amd64-cpython-312\gensim\corpora
      copying gensim\corpora\csvcorpus.py -> bui

In [4]:
#Installation de funcy numexpr et pyLDAvis car pyLDAvis 3.4.1 nécessite les packages funcy et numexpr, qui ne sont pas installés.

!pip install funcy numexpr
!pip install pyLDAvis


  Using cached funcy-2.0-py2.py3-none-any.whl (30 kB)
   ---------------------------------------- 0.0/95.6 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/95.6 kB ? eta -:--:--
   ------------ --------------------------- 30.7/95.6 kB 325.1 kB/s eta 0:00:01
   ----------------- ---------------------- 41.0/95.6 kB 279.3 kB/s eta 0:00:01
   ------------------------------ --------- 71.7/95.6 kB 357.2 kB/s eta 0:00:01
   ---------------------------------------- 95.6/95.6 kB 420.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 MB 262.6 kB/s eta 0:00:10
      --------------------------------------- 0.1/2.6 MB 469.7 kB/s eta 0:00:06
     -- ------------------------------------- 0.2/2.6 MB 838.4 kB/s eta 0:00:03
     --- ------------------------------------ 0.3/2.6 MB 1.1 MB/s eta 0:00:03
     ----- --------------

  error: subprocess-exited-with-error
  
  × Building wheel for gensim (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [714 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\gensim
      copying gensim\downloader.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\interfaces.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\matutils.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\nosy.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\utils.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\__init__.py -> build\lib.win-amd64-cpython-312\gensim
      creating build\lib.win-amd64-cpython-312\gensim\corpora
      copying gensim\corpora\bleicorpus.py -> build\lib.win-amd64-cpython-312\gensim\corpora
      copying gensim\corpora\csvcorpus.py -> bui

In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
#Charger le dataset, ici children_covid_analyse.csv
def load_data(path):
    return pd.read_csv(path)
tweets_df=load_data("../../Donnees/analyse_texte/csv/children_covid_analyse.csv") 
#Pour effectuer un autre TM il faudra juste changer le nom du dataset

In [9]:
#La racinisation (stemming) consiste à transformer les mots en forme radicale c’est ce que fait la fonction SnowballStemmer de Nltk.
stemmer = SnowballStemmer('english') #ici anglais car nos tweets sont en anglais

In [10]:
tweets_df['Texte corrigé'].dtype

dtype('O')

In [11]:
#Toutes les lignes de la colonne "Texte corrigé" n'étaient pas toutes de types 'str'
tweets_df['Texte corrigé'] = tweets_df['Texte corrigé'].astype(str)

#Suppression des liens commençant par http et https  
def remove_url(text):
 return re.sub(r'https?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url)
def remove_url1(text):
 return re.sub(r'http?:\S*', '', text)
tweets_df['Texte corrigé']=tweets_df['Texte corrigé'].apply(remove_url1)

In [12]:
#opération de mettre les mots sous leur forme canonique 
#ex: running devient run 
def lemmatize_stemming(text) :
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))

In [13]:
#suppression des stopwords (the, or, and…) et des mots de moins de 3 lettres
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


# Imprimez le résultat intermédiaire pour chaque document
processed_docs = [preprocess(doc) for doc in tweets_df]
for doc in processed_docs:
    # Concaténez les mots pour former une phrase
    doc_text = ' '.join(doc)
    print(preprocess(doc_text))
    



['fichier']
[]
['retweet']
['like']
['répon']
['citat']
['hashtag']
['text']
[]
['phrase']
[]
['sentiment']
['polarité']
['subjectivité']
['phrase', 'nomin']
['text', 'corrigé']
['text', 'tokénizé']


In [14]:
# On stocke les données dans un dictionnaire Gensim et on les convertit au format Bag Of Words(bow).
#On a alors un couple mot/nb d'occurence

# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Créez le dictionnaire et le corpus
dictionary = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Créez et entraînez le modèle LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, # Le nombre de topics à extraire
                                       id2word=dictionary, # Le mapping entre les identifiants et les mots
                                       passes=2000) # Le nombre d'itérations dans le corpus/ passages




# Affichez les topics
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)


Topic: 0 -> Words: 0.069*"vaccin" + 0.063*"covid" + 0.055*"child" + 0.041*"kiss" + 0.031*"coronavirus" + 0.027*"finer" + 0.021*"corona" + 0.021*"biontech" + 0.014*"day" + 0.013*"impfung"
Topic: 1 -> Words: 0.013*"event" + 0.008*"institut" + 0.008*"serum" + 0.008*"virtual" + 0.008*"poll" + 0.008*"particip" + 0.007*"print" + 0.006*"accept" + 0.005*"parent" + 0.005*"want"
Topic: 2 -> Words: 0.067*"vaccin" + 0.066*"covid" + 0.055*"child" + 0.014*"coronavirus" + 0.012*"trial" + 0.009*"cdcgov" + 0.009*"pidsocieti" + 0.009*"realtimecovid" + 0.009*"idsainfo" + 0.009*"astrazeneca"
Topic: 3 -> Words: 0.087*"vaccin" + 0.067*"child" + 0.059*"covid" + 0.010*"modern" + 0.008*"coronavirus" + 0.008*"school" + 0.006*"begin" + 0.006*"student" + 0.006*"follow" + 0.006*"test"
Topic: 4 -> Words: 0.042*"vaccin" + 0.035*"child" + 0.033*"covid" + 0.030*"help" + 0.025*"money" + 0.024*"famili" + 0.023*"hopeless" + 0.022*"financ" + 0.021*"covid_" + 0.020*"christian"
Topic: 5 -> Words: 0.033*"vaccin" + 0.028*"chi

temps d'exécution pour 1407 lignes et 1000 passages: 10 min 30.1 s 
pour 2000 passages: 21 min 58.6 s 

In [15]:
#Mesure de la cohérence du modèle
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.36684577502909327


Pour 1000 passages et 10 topics on a une cohérence de 0.40353069757626814
Pour 2000 passages et 10 topics  on a une cohérence de 0.41939273995841353

On observe aucune augmentation significative. 
La cohérence du modèle LDA doit être comprise entre 0.4 et 0.7. Dans la documentation il est proposé d'augmenter le nb de passages à 10000 pour avoir un score proche de 70%


In [16]:
#On garde les résultats dans un df
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [17]:
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.069, vaccin)","(0.063, covid)","(0.055, child)","(0.041, kiss)","(0.031, coronavirus)","(0.027, finer)","(0.021, corona)","(0.021, biontech)","(0.014, day)","(0.013, impfung)"
Topic 2,"(0.013, event)","(0.008, institut)","(0.008, serum)","(0.008, virtual)","(0.008, poll)","(0.008, particip)","(0.007, print)","(0.006, accept)","(0.005, parent)","(0.005, want)"
Topic 3,"(0.067, vaccin)","(0.066, covid)","(0.055, child)","(0.014, coronavirus)","(0.012, trial)","(0.009, cdcgov)","(0.009, pidsocieti)","(0.009, realtimecovid)","(0.009, idsainfo)","(0.009, astrazeneca)"
Topic 4,"(0.087, vaccin)","(0.067, child)","(0.059, covid)","(0.010, modern)","(0.008, coronavirus)","(0.008, school)","(0.006, begin)","(0.006, student)","(0.006, follow)","(0.006, test)"
Topic 5,"(0.042, vaccin)","(0.035, child)","(0.033, covid)","(0.030, help)","(0.025, money)","(0.024, famili)","(0.023, hopeless)","(0.022, financ)","(0.021, covid_)","(0.020, christian)"
Topic 6,"(0.033, vaccin)","(0.028, child)","(0.026, covid)","(0.007, guardian)","(0.007, late)","(0.007, carlosdelrio)","(0.006, world)","(0.006, widen)","(0.005, account)","(0.005, consent)"
Topic 7,"(0.113, vaccin)","(0.090, child)","(0.084, covid)","(0.035, finer)","(0.026, age)","(0.020, year)","(0.016, approv)","(0.010, coronavirus)","(0.009, author)","(0.009, biontech)"
Topic 8,"(0.101, vaccin)","(0.083, covid)","(0.074, child)","(0.023, kiss)","(0.017, health)","(0.016, anaem)","(0.012, school)","(0.011, coronavirus)","(0.011, parent)","(0.009, move)"
Topic 9,"(0.070, vaccin)","(0.063, covid)","(0.051, child)","(0.016, watch)","(0.014, lid)","(0.012, question)","(0.012, share)","(0.012, health)","(0.011, fear)","(0.010, downhil)"
Topic 10,"(0.067, covid)","(0.046, vaccin)","(0.043, child)","(0.021, australia)","(0.018, head)","(0.015, move)","(0.012, israel)","(0.012, protect)","(0.010, scienc)","(0.009, covidisnotov)"


In [18]:
import pyLDAvis.gensim_models

In [19]:
#Visu interractive affichant les topics, les mots etc
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.187078  0.023781       1        1  31.710168
7     -0.100344 -0.012556       2        1  27.661493
2     -0.080158 -0.004033       3        1   8.955205
8     -0.010854 -0.053892       4        1   6.694094
3     -0.045952 -0.015060       5        1   6.222488
9      0.018146 -0.000040       6        1   5.913396
0     -0.047965 -0.014525       7        1   4.582003
4      0.123034  0.218692       8        1   4.545605
5      0.108110 -0.049962       9        1   2.357051
1      0.223063 -0.092406      10        1   1.358496, topic_info=         Term         Freq        Total Category  logprob  loglift
15     vaccin  2024.000000  2024.000000  Default  30.0000  30.0000
3       covid  1654.000000  1654.000000  Default  29.0000  29.0000
1       child  1571.000000  1571.000000  Default  28.0000  28.0000
62       kiss   275.000000   275.000000  Default  27.0000  27.0000
47      finer   317.000000   317.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
596  restrict     1.036268     5.568379  Topic10  -5.7012   2.6173
624      want     1.595599    21.302379  Topic10  -5.2696   1.7072
65     parent     1.697788   129.468920  Topic10  -5.2075  -0.0353
287  question     1.343505    37.097378  Topic10  -5.4415   0.9805
164    famili     1.214892    44.965629  Topic10  -5.5422   0.6875

[586 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2593      4  0.806863    absolut
1313      1  0.457119     accept
1313     10  0.457119     accept
1612      6  0.227227    account
1612      9  0.681681    account
...     ...       ...        ...
77        4  0.020397      young
976       1  0.940917    younger
1165      3  0.756029  youngster
1165      5  0.168006  youngster
1165      6  0.084003  youngster

[999 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 3, 9, 4, 10, 1, 5, 6, 2])

Attribution des Topics à chaque tweets

In [20]:
# Utilisez la colonne "Texte corrigé" pour le prétraitement
processed_docs = [preprocess(doc) for doc in tweets_df['Texte corrigé']]

# Obtenez la distribution des sujets pour chaque tweet
tweet_topic_distributions = [lda_model.get_document_topics(dictionary.doc2bow(doc)) for doc in processed_docs]


In [21]:
# Choisissez le sujet dominant pour chaque tweet
dominant_topics = [max(topic_dist, key=lambda x: x[1])[0] for topic_dist in tweet_topic_distributions]

# Ajoutez la colonne 'Dominant Topic' au DataFrame
tweets_df['Dominant Topic'] = dominant_topics


In [22]:
# Créer une Mapping pour les Topics
topic_mapping = {}
for i in range(lda_model.num_topics):
    topic_mapping[i] = 'Topic {}'.format(i+1)


In [23]:
tweets_df['Topic Label'] = [topic_mapping[dominant_topic] for dominant_topic in dominant_topics]


In [25]:
# Affichez les premières lignes du DataFrame
tweets_df.head()


,Fichier,ID,Nb retweet,Nb like,Nb réponses,Nb citations,Hashtags,Texte,Mots,Phrases,Tags POS,Sentiment,Polarité,Subjectivité,Phrases nominales,Texte corrigé,Texte tokénizé,Dominant Topic,Topic Label
0,1432163370669596672.json,"1,4321633706696E+018",0,1,1,0.0,"['#VACCINE', '#COVID19', '#CHILDREN', '#MASK']",Data doesn't lie. This virus does not effect c...,"['Data', 'does', ""n't"", 'lie', 'This', 'virus'...","[Sentence(""Data doesn't lie.""), Sentence(""This...","[('Data', 'NNP'), ('does', 'VBZ'), (""n't"", 'RB...","Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.000000,"['data', ""n't lie"", 'effect children', '# vacc...",Data doesn't lie. His virus does not effect ch...,"['Data', 'does', ""n't"", 'lie', '.', 'This', 'v...",6,Topic 7
1,1361269067441713153.json,"1,36126906744171E+018",0,0,0,0.0,"['#COVID19', '#CORONAVIRUS', '#HEALTH', '#HEAL...",A new trial is to test how well the Oxford-Ast...,"['A', 'new', 'trial', 'is', 'to', 'test', 'how...","[Sentence(""A new trial is to test how well the...","[('A', 'DT'), ('new', 'JJ'), ('trial', 'NN'), ...","Sentiment(polarity=0.018181818181818174, subje...",0.018182,0.427273,"['new trial', 'oxford-astrazeneca', 'coronavir...",A new trial is to test how well the Oxford-Ast...,"['A', 'new', 'trial', 'is', 'to', 'test', 'how...",7,Topic 8
2,1382650191526965249.json,"1,38265019152697E+018",0,2,0,0.0,"['#CHILDREN', '#KINDER', '#CORONAVIRUSDE', '#C...",'Kids are going to be the key' to get to herd ...,"[""'Kids"", 'are', 'going', 'to', 'be', 'the', '...","[Sentence(""'Kids are going to be the key' to g...","[(""'Kids"", 'NNS'), ('are', 'VBP'), ('going', '...","Sentiment(polarity=0.0, subjectivity=1.0)",0.000000,1.000000,"['herd immunity # children #', 'kinder', 'karl...",'Lids are going to be the key' to get to herd ...,"[""'Kids"", 'are', 'going', 'to', 'be', 'the', '...",0,Topic 1
3,1436717418840088582.json,"1,43671741884009E+018",0,0,0,0.0,"['#SOUTHAFRICA', '#HEALTH', '#PFIZER', '#COVID...",#SouthAfrica's #health regulator has approved ...,"['SouthAfrica', ""'s"", 'health', 'regulator', '...","[Sentence(""#SouthAfrica's #health regulator ha...","[('SouthAfrica', 'NNP'), (""'s"", 'POS'), ('heal...","Sentiment(polarity=0.18888888888888888, subjec...",0.188889,0.411111,"['southafrica', ""'s # health regulator"", 'pfiz...",#SouthAfrica's #health regulatory has approved...,"['#', 'SouthAfrica', ""'s"", '#', 'health', 'reg...",6,Topic 7
4,1456708686437621764.json,"1,45670868643762E+018",0,0,0,0.0,"['#CHICAGO', '#VACCINE', '#CPS', '#SCHOOL', '#...","""With a week’s notice, #Chicago Public Schools...","['With', 'a', 'week', '’', 's', 'notice', 'Chi...","[Sentence(""""With a week’s notice, #Chicago Pub...","[('With', 'IN'), ('a', 'DT'), ('week', 'NN'), ...","Sentiment(polarity=0.0, subjectivity=0.0666666...",0.000000,0.066667,"['week ’ s notice', 'chicago', 'public schools...","""With a week’s notice, #Chicago Public Schools...","['``', 'With', 'a', 'week', '’', 's', 'notice'...",5,Topic 6
